In [1]:
%matplotlib inline


# 3D scattering quantum chemistry regression

Description:
This example trains a classifier combined with a scattering transform to
regress molecular atomization energies on the QM7 dataset. Here, we use full
charges, valence charges and core charges. A linear regression is deployed.

Remarks:
The linear regression of the QM7 energies with the given values gives MAE
2.75, RMSE 4.18 (kcal.mol-1)

Reference:
https://arxiv.org/abs/1805.00571


## Preliminaries

First, we import NumPy, PyTorch, and some utility modules.



In [1]:
import numpy as np
import torch
import time
import os

We will use scikit-learn to construct a linear model, so we import the
necessary modules. In addition, we need to compute distance matrices when
normalizing our input features, so we import `pdist` from `scipy.spatial`.



In [2]:
from sklearn import (linear_model, model_selection, preprocessing,
                     pipeline)
from scipy.spatial.distance import pdist

We then import the necessary functionality from Kymatio. First, we need the
PyTorch frontend of the 3D solid harmonic cattering transform.



In [3]:
from kymatio.torch import HarmonicScattering3D

The 3D transform doesn't compute the zeroth-order coefficients, so we need
to import `compute_integrals` to do this manually.



In [4]:
from kymatio.scattering3d.backend.torch_backend \
    import TorchBackend3D

To generate the input 3D maps, we need to calculate sums of Gaussians, so we
import the function `generate_weighted_sum_of_gaussians`.



In [5]:
from kymatio.scattering3d.utils \
    import generate_weighted_sum_of_gaussians

Finally, we import the utility functions that let us access the QM7 dataset
and the cache directories to store our results.



In [6]:
from kymatio.datasets import fetch_qm7
from kymatio.caching import get_cache_dir

## Data preparation

Fetch the QM7 database and extract the atomic positions and nuclear charges
of each molecule. This dataset contains 7165 organic molecules with up to
seven non-hydrogen atoms, whose energies were computed using density
functional theory.



In [7]:
import sys
sys.path.append(os.path.abspath(".."))
from utils_project import generate_csv,create_dataframe_from_xyz_files,create_X_y_from_dataframe


csv_path = "../../data/energies/train.csv"
path_data = "../../data/atoms/train"
df_train=create_dataframe_from_xyz_files(path_data,csv_path)
X=df_train[['positions', 'energy', 'charges']]

qm7 = X.to_dict("list")

#qm7 = fetch_qm7(align=True)
pos = np.array(qm7['positions'])
full_charges = np.array(qm7['charges'])

n_molecules = pos.shape[0]

In [8]:
for clé, valeur in qm7.items():
    print(f"Clé: {clé}, Haut de la liste: {valeur[:30]}")

print(qm7.keys())

Clé: positions, Haut de la liste: [array([[ 0.459861, -1.449313,  0.324746],
       [ 0.084621,  0.030369,  0.16977 ],
       [-1.375102,  0.206983, -0.004939],
       [-2.169283, -0.104147,  1.181523],
       [ 0.804741,  0.662336, -1.040134],
       [ 2.213035,  0.645181, -0.87253 ],
       [ 1.5416  , -1.560619,  0.339062],
       [ 0.069454, -2.036593, -0.503465],
       [ 0.063565, -1.864862,  1.248989],
       [ 0.430383,  0.569359,  1.088783],
       [-1.565395,  1.16976 , -0.295629],
       [-2.122486, -1.172491,  1.407793],
       [-3.212633,  0.157544,  0.994324],
       [-1.834786,  0.443019,  2.081621],
       [ 0.451408,  1.708129, -1.181228],
       [ 0.575972,  0.115237, -1.965693],
       [ 2.444715,  1.025713, -0.015171],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ]]

From the nuclear charges, we compute the number of valence electrons, which
we store as the valence charge of that atom.



In [9]:
mask = full_charges <= 2
valence_charges = full_charges * mask

mask = np.logical_and(full_charges > 2, full_charges <= 10)
valence_charges += (full_charges - 2) * mask

mask = np.logical_and(full_charges > 10, full_charges <= 18)
valence_charges += (full_charges - 10) * mask

We then normalize the positions of the atoms. Specifically, the positions
are rescaled such that two Gaussians of width `sigma` placed at those
positions overlap with amplitude less than `overlapping_precision`.



In [10]:
overlapping_precision = 1e-1
sigma = 2.0
min_dist = np.inf

for i in range(n_molecules):
    n_atoms = np.sum(full_charges[i] != 0)
    pos_i = pos[i, :n_atoms, :]
    min_dist = min(min_dist, pdist(pos_i).min())

delta = sigma * np.sqrt(-8 * np.log(overlapping_precision))
pos = pos * delta / min_dist

## Scattering Transform
Given the rescaled positions and charges, we are now ready to compute the
density maps by placing Gaussians at the different positions weighted by the
appropriate charge. These are fed into the 3D solid harmonic scattering
transform to obtain features that are used to regress the energies. In
order to do this, we must first define a grid.



In [12]:
M, N, O = 32, 32, 32 #192, 128, 96
grille = "32-32-32"
grid = np.mgrid[-M//2:-M//2+M, -N//2:-N//2+N, -O//2:-O//2+O]
grid = np.fft.ifftshift(grid)

We then define the scattering transform using the `HarmonicScattering3D`
class.



In [13]:
J = 2
L = 3
integral_powers = [0.5, 1.0, 2.0, 3.0]

scattering = HarmonicScattering3D(J=J, shape=(M, N, O),
                                  L=L, sigma_0=sigma,
                                  integral_powers=integral_powers)

We then check whether a GPU is available, in which case we transfer our
scattering object there.



In [14]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
scattering.to(device)
print(device)

cuda


The maps computed for each molecule are quite large, so the computation has
to be done by batches. Here we select a small batch size to ensure that we
have enough memory when running on the GPU. Dividing the number of molecules
by the batch size then gives us the number of batches.



In [15]:
batch_size = 8
n_batches = int(np.ceil(n_molecules / batch_size))

We are now ready to compute the scattering transforms. In the following
loop, each batch of molecules is transformed into three maps using Gaussians
centered at the atomic positions, one for the nuclear charges, one for the
valence charges, and one with their difference (called the “core” charges).
For each map, we compute its scattering transform up to order two and store
the results.



In [16]:
order_0, orders_1_and_2 = [], []
print('Computing solid harmonic scattering coefficients of '
      '{} molecules from the QM7 database on {}'.format(
        n_molecules, "GPU" if use_cuda else "CPU"))
print('sigma: {}, L: {}, J: {}, integral powers: {}'.format(
        sigma, L, J, integral_powers))

this_time = None
last_time = None

for i in range(n_batches):
    this_time = time.time()
    if last_time is not None:
        dt = this_time - last_time
        print("Iteration {} ETA: [{:02}:{:02}:{:02}]".format(
            i + 1, int(((n_batches - i - 1) * dt) // 3600),
            int((((n_batches - i - 1) * dt) // 60) % 60),
            int(((n_batches - i - 1) * dt) % 60)))
    else:
        print("Iteration {} ETA: {}".format(i + 1, '-'))
    last_time = this_time
    time.sleep(1)

    # Extract the current batch.
    start = i * batch_size
    end = min(start + batch_size, n_molecules)

    pos_batch = pos[start:end]
    full_batch = full_charges[start:end]
    val_batch = valence_charges[start:end]

    # === Nuclear density ===
    full_density_batch = generate_weighted_sum_of_gaussians(grid,
                                pos_batch, full_batch, sigma)
    full_density_batch = torch.from_numpy(full_density_batch).float().to(device)

    full_order_0 = TorchBackend3D.compute_integrals(full_density_batch, integral_powers)
    full_scattering = scattering(full_density_batch)

    # === Valence density ===
    val_density_batch = generate_weighted_sum_of_gaussians(grid,
                                pos_batch, val_batch, sigma)
    val_density_batch = torch.from_numpy(val_density_batch).float().to(device)

    val_order_0 = TorchBackend3D.compute_integrals(val_density_batch, integral_powers)
    val_scattering = scattering(val_density_batch)

    # === Core density ===
    core_density_batch = full_density_batch - val_density_batch
    core_order_0 = TorchBackend3D.compute_integrals(core_density_batch, integral_powers)
    core_scattering = scattering(core_density_batch)

    # === Stack all coefficients ===
    batch_order_0 = torch.stack((full_order_0, val_order_0, core_order_0), dim=-1).to(device)
    batch_orders_1_and_2 = torch.stack((full_scattering, val_scattering, core_scattering), dim=-1).to(device)

    order_0.append(batch_order_0)
    orders_1_and_2.append(batch_orders_1_and_2)


Computing solid harmonic scattering coefficients of 6591 molecules from the QM7 database on GPU
sigma: 2.0, L: 3, J: 2, integral powers: [0.5, 1.0, 2.0, 3.0]
Iteration 1 ETA: -
Iteration 2 ETA: [00:17:44]
Iteration 3 ETA: [00:16:18]
Iteration 4 ETA: [00:16:23]
Iteration 5 ETA: [00:16:16]
Iteration 6 ETA: [00:16:17]
Iteration 7 ETA: [00:16:25]
Iteration 8 ETA: [00:16:14]
Iteration 9 ETA: [00:16:16]
Iteration 10 ETA: [00:16:12]
Iteration 11 ETA: [00:16:09]
Iteration 12 ETA: [00:16:11]
Iteration 13 ETA: [00:16:07]
Iteration 14 ETA: [00:16:16]
Iteration 15 ETA: [00:16:10]
Iteration 16 ETA: [00:16:03]
Iteration 17 ETA: [00:16:09]
Iteration 18 ETA: [00:16:04]
Iteration 19 ETA: [00:16:05]
Iteration 20 ETA: [00:16:09]
Iteration 21 ETA: [00:15:58]
Iteration 22 ETA: [00:16:00]
Iteration 23 ETA: [00:15:57]
Iteration 24 ETA: [00:15:48]
Iteration 25 ETA: [00:15:52]
Iteration 26 ETA: [00:15:56]
Iteration 27 ETA: [00:15:50]
Iteration 28 ETA: [00:16:03]
Iteration 29 ETA: [00:15:53]
Iteration 30 ETA: [

In [17]:
# Fusionner les résultats en un seul tenseur
order_0_tensor = torch.cat(order_0, dim=0)
orders_1_and_2_tensor = torch.cat(orders_1_and_2, dim=0)

# Sauvegarder les tenseurs sur disque
torch.save({
    'order_0': order_0_tensor,
    'orders_1_and_2': orders_1_and_2_tensor
}, f'../models_scattering/scattering_outputs_{grille}.pt')

In [18]:
# Charger les données
saved_data = torch.load( f'../models_scattering/scattering_outputs_{grille}.pt', map_location=device)
order_0 = saved_data['order_0']
orders_1_and_2 = saved_data['orders_1_and_2']

Concatenate the batch outputs and transfer to NumPy.



In [19]:
#order_0 = torch.cat(order_0, dim=0)
#orders_1_and_2 = torch.cat(orders_1_and_2, dim=0)

order_0 = order_0.cpu().numpy()
orders_1_and_2 = orders_1_and_2.cpu().numpy()

## Regression

To use the scattering coefficients as features in a scikit-learn pipeline,
these must be of shape `(n_samples, n_features)`, so we reshape our arrays
accordingly.



In [20]:
order_0 = order_0.reshape((n_molecules, -1))
orders_1_and_2 = orders_1_and_2.reshape((n_molecules, -1))

Since the above calculation is quite lengthy, we save the results to a cache
for future use.



In [21]:
basename = 'qm7_L_{}_J_{}_sigma_{}_MNO_{}_powers_{}.npy'.format(
        L, J, sigma, (M, N, O), integral_powers)

cache_dir = get_cache_dir("qm7/experiments")

filename = os.path.join(cache_dir, 'order_0_' + basename)
np.save(filename, order_0)

filename = os.path.join(cache_dir, 'orders_1_and_2' + basename)
np.save(filename, orders_1_and_2)

We now concatenate the zeroth-order coefficients with the rest since we want
to use all of them as features.



In [22]:
scattering_coef = np.concatenate([order_0, orders_1_and_2], axis=1)

Fetch the target energies from the QM7 dataset.



In [23]:
target = qm7['energy']

We evaluate the performance of the regression using five-fold
cross-validation. To do so, we first shuffle the molecules, then we store
the resulting indices in `cross_val_folds`.



In [24]:
n_folds = 3

P = np.random.permutation(n_molecules).reshape((n_folds, -1))

cross_val_folds = []

for i_fold in range(n_folds):
    fold = (np.concatenate(P[np.arange(n_folds) != i_fold], axis=0),
            P[i_fold])
    cross_val_folds.append(fold)

Given these folds, we compute the regression error for various settings of
the `alpha` parameter, which controls the amount of regularization applied
to the regression problem (here in the form of a simple ridge regression, or
Tikhonov, regularization). The mean absolute error (MAE) and root mean
square error (RMSE) is output for each value of `alpha`.



In [25]:
import numpy as np
from sklearn import linear_model, preprocessing, pipeline, model_selection
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
import joblib

# Supposons que scattering_coef et target soient déjà définis
cross_val_folds = 5  # Assurez-vous que cross_val_folds est défini

# Liste des modèles à tester
models = [
    ("Ridge Regression with alpha=0.1", linear_model.Ridge(alpha=0.1)),
    ("Ridge Regression with alpha=1", linear_model.Ridge(alpha=1)),
    ("Ridge Regression with alpha=10", linear_model.Ridge(alpha=10)),
    ("Lasso Regression", linear_model.Lasso()),
    ("ElasticNet Regression", linear_model.ElasticNet()),
    ("Random Forest Regression", RandomForestRegressor()),
    ("Support Vector Regression", SVR()),
    ("XGBoost Regression", XGBRegressor()),
    ("MLP Regressor", MLPRegressor(random_state=1, max_iter=2000, tol=0.1))
]

results = []

for name, model in models:
    scaler = preprocessing.StandardScaler()
    regressor = pipeline.make_pipeline(scaler, model)

    target_prediction = model_selection.cross_val_predict(regressor, X=scattering_coef, y=target, cv=cross_val_folds)

    MAE = np.mean(np.abs(target_prediction - target))
    RMSE = np.sqrt(np.mean((target_prediction - target) ** 2))

    results.append((name, model, MAE, RMSE))

    print('{}: MAE: {}, RMSE: {}'.format(name, MAE, RMSE))

# Trouver le modèle avec le RMSE le plus bas
best_result = min(results, key=lambda x: x[3])
best_model_name, best_model, best_mae, best_rmse = best_result

print(f"Le meilleur modèle est {best_model_name} avec un RMSE de {best_rmse}.")

/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.3425e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.3768e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.39341e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.45626e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/gris/miniforge3/lib/python3.10/site-pack

Ridge Regression with alpha=0.1: MAE: 5.2974951506572445, RMSE: 6.899245360079598
Ridge Regression with alpha=1: MAE: 5.501608469712773, RMSE: 6.970971018185825
Ridge Regression with alpha=10: MAE: 5.706981861320807, RMSE: 7.226472138578132
Lasso Regression: MAE: 7.289267452599622, RMSE: 9.01671955474425
ElasticNet Regression: MAE: 7.090911263889433, RMSE: 8.814307874821802
Random Forest Regression: MAE: 6.030557161453029, RMSE: 7.706526384425391
Support Vector Regression: MAE: 6.514683392305584, RMSE: 8.317423353852755
XGBoost Regression: MAE: 6.0430533071297186, RMSE: 7.819945338140339
MLP Regressor: MAE: 5.979982297555173, RMSE: 7.553237234713717
Le meilleur modèle est Ridge Regression with alpha=0.1 avec un RMSE de 6.899245360079598.


In [26]:
# Entraîner le meilleur modèle sur l'ensemble des données
scaler = preprocessing.StandardScaler()
best_regressor = pipeline.make_pipeline(scaler, best_model)
best_regressor.fit(scattering_coef, target)

# Enregistrer le meilleur modèle
joblib.dump(best_regressor, f'../models_scattering/best_model_{grille}.pkl')

/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=1.80347e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


['../models_scattering/best_model_32-32-32.pkl']